In [5]:
import requests
import zipfile
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import altair as alt

In [6]:
zip_url_1= "https://github.com/aaubs/ds-master/raw/main/data/assignments_datasets/KIVA/kiva_loans_part_0.csv.zip"
zip_url_2= "https://github.com/aaubs/ds-master/raw/main/data/assignments_datasets/KIVA/kiva_loans_part_1.csv.zip"
zip_url_3= "https://github.com/aaubs/ds-master/raw/main/data/assignments_datasets/KIVA/kiva_loans_part_2.csv.zip"

    # Download the ZIP file
response = requests.get(zip_url_1)
response.raise_for_status()  # Check if the request was successful

# Open the ZIP file from the response content
with zipfile.ZipFile(io.BytesIO(response.content)) as zf:
     # List all files in the ZIP
    print(zf.namelist())

        # Read a specific CSV file from the ZIP
    df1 = pd.read_csv(zf.open('kiva_loans_part_0.csv'))


    # Download the ZIP file
response = requests.get(zip_url_2)
response.raise_for_status()  # Check if the request was successful

    # Open the ZIP file from the response content
with zipfile.ZipFile(io.BytesIO(response.content)) as zf:
        # List all files in the ZIP
    print(zf.namelist())

        # Read a specific CSV file from the ZIP
    df2 = pd.read_csv(zf.open('kiva_loans_part_1.csv'))

    # Download the ZIP file
response = requests.get(zip_url_3)
response.raise_for_status()  # Check if the request was successful

    # Open the ZIP file from the response content
with zipfile.ZipFile(io.BytesIO(response.content)) as zf:
        # List all files in the ZIP
    print(zf.namelist())

        # Read a specific CSV file from the ZIP
    df3 = pd.read_csv(zf.open('kiva_loans_part_2.csv'))



data = pd.concat([df1, df2, df3])


['kiva_loans_part_0.csv', '__MACOSX/._kiva_loans_part_0.csv']
['kiva_loans_part_1.csv', '__MACOSX/._kiva_loans_part_1.csv']
['kiva_loans_part_2.csv', '__MACOSX/._kiva_loans_part_2.csv']


In [7]:
data.drop(['tags','posted_time','disbursed_time', 'funded_time', 'date', 'country_code' , 'use', 'region', 'currency'], axis = 'columns', inplace = True)
data.dropna(inplace = True)

valid_genders = ['male', 'female']
data = data[data['borrower_genders'].isin(valid_genders)]
data_small = data.iloc[0:35000]
data_small.reset_index(drop=True)
data_small

,id,funded_amount,loan_amount,activity,sector,country,partner_id,term_in_months,lender_count,borrower_genders,repayment_interval
0,653051,300.0,300.0,Fruits & Vegetables,Food,Pakistan,247.0,12.0,12,female,irregular
2,653068,150.0,150.0,Transportation,Transportation,India,334.0,43.0,6,female,bullet
3,653063,200.0,200.0,Embroidery,Arts,Pakistan,247.0,11.0,8,female,irregular
4,653084,400.0,400.0,Milk Sales,Food,Pakistan,245.0,14.0,16,female,monthly
6,653067,200.0,200.0,Dairy,Agriculture,India,334.0,43.0,8,female,bullet
...,...,...,...,...,...,...,...,...,...,...,...
41544,694659,500.0,500.0,Grocery Store,Food,Zimbabwe,305.0,8.0,14,female,irregular
41545,694478,425.0,425.0,General Store,Retail,Pakistan,245.0,14.0,17,female,monthly
41547,694167,325.0,325.0,Tailoring,Services,Pakistan,247.0,11.0,11,female,irregular
41548,694444,2500.0,2500.0,Farming,Agriculture,Armenia,169.0,26.0,61,female,monthly


In [8]:
#Step 1: Label Encoding and Matrix Creation
# # A. Label Encoding
# import numpy as np
# from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder
le_sector = LabelEncoder()
le_userid = LabelEncoder()
data['sector_id'] = le_sector.fit_transform(data['sector'])
data['user_id'] = le_userid.fit_transform(data['id'])

data_small = data.iloc[0:35000]
data_small.reset_index(drop=True)


# B. Matrix Creation
matrix_user_loan = data_small.groupby(['user_id','sector_id'])['funded_amount'].agg('mean').unstack().fillna(0)

#Step 2: Perform Dimensionality Reduction
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
matrix_user_loan_dr = svd.fit_transform(matrix_user_loan)

#Step 3: Calculate the Similarity Matrix
from sklearn.metrics.pairwise import cosine_distances
cosine_distances_matrix_user_loan_dr = cosine_distances(matrix_user_loan_dr)

We selected the user with ID 653067 and employed the cosine similarity matrix to identify the individuals most similar to them.




In [9]:
le_userid.transform(['653067'])

array([16])

In [10]:
le_userid.inverse_transform([16])

array([653067])

In [11]:
data_small.loc[data_small['id'] == 653067]

,id,funded_amount,loan_amount,activity,sector,country,partner_id,term_in_months,lender_count,borrower_genders,repayment_interval,sector_id,user_id
6,653067,200.0,200.0,Dairy,Agriculture,India,334.0,43.0,8,female,bullet,0,16


In [12]:
np.argsort(cosine_distances_matrix_user_loan_dr[16,:])[:5]

array([17499, 10342, 27526, 27524, 10355])

In [13]:
le_userid.inverse_transform([17499])

array([673478])

In [14]:
data_small.loc[data_small['id'] == 673478]

,id,funded_amount,loan_amount,activity,sector,country,partner_id,term_in_months,lender_count,borrower_genders,repayment_interval,sector_id,user_id
20399,673478,125.0,125.0,Farming,Agriculture,Philippines,136.0,8.0,4,male,irregular,0,17499
